# This is the code for twitter sentiment anylasis 

In [1]:
#importing lab to use
import pandas as pd 
import numpy as np 
import re
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
import xgboost as xbs
from sklearn.tree import DecisionTreeClassifier

In [2]:
train = pd.read_csv("train.csv")# reading csv files 
test = pd.read_csv("test.csv")


In [3]:
train.head() # printing train with first 5 col

,textID,text,selected_text,sentiment
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive


In [4]:
train.shape # printing the shape of train

(27486, 4)

In [5]:
train.columns # printing what rows names

Index(['textID', 'text', 'selected_text', 'sentiment'], dtype='object')

In [6]:
# we are given a data with 1) text id  which is of no use to us 
# 2) text which we will use 
# 3) selected text it is the part which supports the sentiment

In [7]:
# we will first train the model with text then selected text 
# we would get better acc with text as we have more data 

In [8]:
x_train = train["text"] # we are just selecting text for now

In [9]:
x_train.head() # printing x train first 5

0    Spent the entire morning in a meeting w/ a ven...
1        Oh! Good idea about putting them on ice cream
2    says good (or should i say bad?) afternoon!  h...
3           i dont think you can vote anymore! i tried
4               haha better drunken tweeting you mean?
Name: text, dtype: object

In [10]:
y_train = train["sentiment"] # selecting sentiment for y label which we are going to predict

In [11]:
y_train.head() # printing y_train head

0     neutral
1    positive
2     neutral
3    negative
4    positive
Name: sentiment, dtype: object

In [12]:
x_train = pd.DataFrame(x_train) #converting x_train to pandas Dataframe

In [13]:
x_train.head() # now if we print x_train 

,text
0,Spent the entire morning in a meeting w/ a ven...
1,Oh! Good idea about putting them on ice cream
2,says good (or should i say bad?) afternoon! h...
3,i dont think you can vote anymore! i tried
4,haha better drunken tweeting you mean?


In [14]:
test.head() # printing test 

,textID,text,sentiment
0,11aa4945ff,http://twitpic.com/67swx - i wish i was calli...,positive
1,fd1db57dc0,i'm done.haha. HOUSE MD marathon ulet,positive
2,2524332d66,I'm concerned for that family,positive
3,0fb19285b2,HEY GUYS IT'S WORKING NO NEED TO WORRY. i have...,positive
4,e6c9e5e3ab,26th February,neutral


In [15]:
x_test = test["text"]

In [16]:
x_test.head()

0     http://twitpic.com/67swx - i wish i was calli...
1                i'm done.haha. HOUSE MD marathon ulet
2                        I'm concerned for that family
3    HEY GUYS IT'S WORKING NO NEED TO WORRY. i have...
4                                        26th February
Name: text, dtype: object

In [17]:
y_test = test["sentiment"]

In [18]:
x_test = pd.DataFrame(x_test)

In [19]:
# we will first clean the text 

In [20]:
def cleanText(text):
    text = re.sub(r"[^A-Za-z0-9]", " ", str(text))
    text = text.lower()
    text = re.sub(r'[^\x00-\x7F]+','-', text)
    text = re.sub("\S*\d\S*", "", text).strip()
    text = re.sub(r"http\S+", "", str(text))
    text = BeautifulSoup(text , 'lxml').get_text()
                  
   
    
    return text

In [21]:
x_train["text"] = x_train["text"].apply(cleanText) # appling cleanText to all xtrain
x_test["text"] = x_test["text"].apply(cleanText) # same to x test

In [22]:
x_train.head()

,text
0,spent the entire morning in a meeting w a ven...
1,oh good idea about putting them on ice cream
2,says good or should i say bad afternoon h...
3,i dont think you can vote anymore i tried
4,haha better drunken tweeting you mean


In [23]:
x_train = x_train["text"]
x_test = x_test["text"]

In [24]:
x_train.head()

0    spent the entire morning in a meeting w  a ven...
1        oh  good idea about putting them on ice cream
2    says good  or should i say bad   afternoon   h...
3           i dont think you can vote anymore  i tried
4                haha better drunken tweeting you mean
Name: text, dtype: object

In [25]:
# now we will train a model

In [26]:
# in machine learning we need to first vectorize the data and we can train a model and vectorize it in just a line

first we will use navie bayes algo

In [27]:

text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', MultinomialNB()) ])

In [28]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [29]:
predicted = text_clf.predict(x_test)

In [30]:
print('Accuracy achieved by navie bayes is  ' + str(np.mean(predicted == y_test)))

Accuracy achieved by navie bayes is  0.6285714285714286


In [31]:
#we got a accuracy of 62 % with the model 

In [32]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', SVC()) ])

In [33]:
# we will now use svm with default parametes

In [34]:
text_clf.fit(x_train,y_train) 

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degr

In [35]:
predicted = text_clf.predict(x_test)

In [36]:
text_clf.predict(["this is a good"]) # test a random sentence

array(['positive'], dtype=object)

In [37]:
print('Accuracy achieved by svm is  ' + str(np.mean(predicted == y_test)))

Accuracy achieved by svm is  0.7142857142857143


In [38]:
# accuaracy achieved by support vector machine is 71%

In [39]:
# we would now use knnn

In [40]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', KNeighborsClassifier()) ])

In [41]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,


In [42]:
predicted = text_clf.predict(x_test)

In [43]:
print('Accuracy achieved by knn  is ' + str(np.mean(predicted == y_test)))

Accuracy achieved by knn  is 0.5207920792079208


In [ ]:
#accuracy by knn is low 52 %

In [44]:
# now lets train logistic regression 

In [45]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', LogisticRegression(class_weight="balanced")) ])

In [46]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                    fit_intercept=True, intercept_sc

In [47]:
predicted = text_clf.predict(x_test)

In [48]:
print('Accuracy achieved by logistic regression  is ' + str(np.mean(predicted == y_test)))

Accuracy achieved by logistic regression  is 0.7057991513437057


In [ ]:
#logistic regression is 70%

In [ ]:
#now lets use random forest

In [53]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', RandomForestClassifier()) ])

In [54]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [55]:
predicted = text_clf.predict(x_test)

In [56]:
print('Accuracy achieved by random forest is ' + str(np.mean(predicted == y_test)))


Accuracy achieved by random forest is 0.68995756718529


In [ ]:
# random forest has 68 % accuricy

In [ ]:
# now lets use decision tree

In [57]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', DecisionTreeClassifier()) ])

In [58]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
               

In [59]:
predicted = text_clf.predict(x_test)

In [60]:
print('Accuracy achieved by decision tree classifier is ' + str(np.mean(predicted == y_test)))

Accuracy achieved by decision tree classifier is 0.6260254596888261


In [ ]:
# it has gieven us 62% 

In [ ]:
# it time for xgboost

In [61]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', xbs.XGBClassifier()) ])

In [62]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                        

In [63]:
predicted = text_clf.predict(x_test)

In [64]:
print('Accuracy achieved by xgboost is ' + str(np.mean(predicted == y_test)))

Accuracy achieved by xgboost is 0.6291371994342292


In [ ]:
#xgboost 62 percent

In [65]:
# we got acc by svm is 71 % and linear regr is 70 and random forest we got 68 
# these are the 3 best models we have 

In [66]:
# lets use svm with linear kernal 
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', SVC(kernel= "linear")) ])

In [67]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degr

In [68]:
predicted = text_clf.predict(x_test)

In [69]:
print('Accuracy achieved by svm linear kernal is ' + str(np.mean(predicted == y_test)))

Accuracy achieved by svm linear kernal is 0.7196605374823196


In [ ]:
# linear kernal svm 71 % 

In [70]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', SVC(kernel= "poly")) ])

In [71]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degr

In [72]:
predicted = text_clf.predict(x_test)

In [73]:
print('Accuracy achieved by svm poly kernal is ' + str(np.mean(predicted == y_test)))

Accuracy achieved by svm poly kernal is 0.5782178217821782


In [74]:

text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', SVC(kernel= "sigmoid")) ])

In [75]:
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degr

In [76]:
predicted = text_clf.predict(x_test)

In [77]:
print('Accuracy achieved by svm sigmoid kernal is ' + str(np.mean(predicted == y_test)))

Accuracy achieved by svm sigmoid kernal is 0.7117397454031117


In [78]:
# let us now u use selected text as x label

In [79]:
train.head()

,textID,text,selected_text,sentiment
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive


In [80]:
sx_train = train["selected_text"]

In [81]:
sx_train = pd.DataFrame(sx_train)

In [82]:
sx_train["selected_text"] = sx_train["selected_text"].apply(cleanText)

In [83]:
sx_train = sx_train["selected_text"]

In [84]:
text_clf = Pipeline([('vect', TfidfVectorizer()), 
                      ('clf', SVC()) ])

In [85]:
text_clf.fit(sx_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degr

In [86]:
predicted = text_clf.predict(x_test)

In [87]:
print('Accuracy achieved by svm  is (used on selected text only) ' + str(np.mean(predicted == y_test)))

Accuracy achieved by svm  is (used on selected text only) 0.5612446958981613


In [88]:
# currently the the acc we can get is 70% with svm 

# Svm and logistic regression is the best model for this 

# we must use svm with "text" not with "selected text"
